In [1]:
import sys
sys.path.append('/g/data1a/e14/as3189/OFAM/scripts/')
import gsw
import numpy as np
import xarray as xr
from scipy import stats
from pathlib import Path
from scipy import interpolate
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.colors import LinearSegmentedColormap

from main_valid import open_tao_data, plot_tao_timeseries, cor_scatter_plot
from main_valid import EUC_depths, plot_eq_velocity
from main import paths, idx_1d, LAT_DEG, lx
# Path to save figures, save data and OFAM model output.
fpath, dpath, xpath, lpath, tpath = paths()


# Plot multiple subplots

In [37]:
T = 1
ds = open_tao_data(frq=lx['frq_short'][T], dz=slice(10, 360))
ds[0].attrs['array']
u = np.ma.masked_invalid(ds[0])
hasattr(u, 'Time')

TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_array()` method.

# Interpolating OFAM3 depth to every 5 m


In [23]:
du = ds[1].u_1205
t = -1
y = du[t].values
x = du[t].depth.values

f = interpolate.interp1d(x, y, kind='slinear')
z = np.arange(10, x[-1]+1, 5)
f(z)

array([        nan,         nan,         nan,         nan, -0.2241077 ,
       -0.10729849, -0.06175052, -0.01560109,  0.03550928,  0.09280071,
        0.16594538,  0.24883835,  0.34268868,  0.4464438 ,  0.55731237,
        0.67626542,  0.78920561,  0.88975161,  0.95700026,  0.99680716,
        1.0050714 ,  0.98722261,  0.95742345,  0.92072505,  0.88559896,
        0.8523742 ,  0.81850386,  0.78312469,  0.74442291,  0.70517695,
        0.66717863,  0.63181657,  0.60003513,  0.5681082 ,  0.53437793,
        0.49205089,  0.44270709,  0.38866565,  0.3367244 ,  0.29194206,
        0.25647962,  0.22806311,  0.20708391,  0.18962742,  0.1756864 ,
        0.16114131,  0.14590985,  0.12908368,  0.11358575,  0.09936476,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,      

In [24]:
files = []
for y in range(lx['years'][0][0], lx['years'][0][1]+1):
    for m in range(1, 13):
        files.append(str(xpath.joinpath('ocean_u_{}_{:02d}.nc'.format(y, m))))

In [27]:
df = xr.open_mfdataset(files, combine='by_coords')

In [369]:
df.st_ocean

<xarray.DataArray 'st_ocean' (st_ocean: 51)>
array([   2.5     ,    7.5     ,   12.5     ,   17.51539 ,   22.667021,
         28.16938 ,   34.218006,   40.954975,   48.454975,   56.718006,
         65.66938 ,   75.167023,   85.015388,   95.      ,  105.      ,
        115.      ,  125.      ,  135.      ,  145.      ,  155.      ,
        165.      ,  175.      ,  185.      ,  195.      ,  205.189896,
        217.054489,  233.194321,  255.884232,  286.608978,  325.884216,
        373.194336,  427.054474,  485.189911,  545.511108,  610.415649,
        685.926758,  775.926758,  880.415649,  995.511108, 1115.313354,
       1238.353882, 1368.157471, 1507.733887, 1658.157471, 1818.353882,
       1985.313354, 2165.180176, 2431.101074, 2894.841797, 3603.101074,
       4509.180176])
Coordinates:
  * st_ocean  (st_ocean) float64 2.5 7.5 12.5 ... 2.895e+03 3.603e+03 4.509e+03
Attributes:
    long_name:       tcell zstar depth
    units:           meters
    cartesian_axis:  Z
    positive:      

In [22]:
df = df.sel(xu_ocean=[165, 190, 220], yu_ocean=slice(-2.6, 2.6), st_ocean=slice(2.5, 485))
# Calculate the monthly mean.
df = ds.resample(Time="MS").mean()
# New depth levels to interpolate to.
z = np.arange(10, 360, 5)


In [23]:
di = df.interp(st_ocean=z, method='slinear')
# di = df.interp(st_ocean=z, method='cubic')

In [21]:
# # Test to chack interpolation depths.
# t = 0
print(df.u[0].sel(xu_ocean=190.,st_ocean=slice(185, 373.194336))[:, 26].values)
print(di.u[0].sel(xu_ocean=190.,st_ocean=slice(10, 355))[:, 26].values)

[0.46271968 0.46586996 0.45597607 0.43525302 0.39988112 0.34530216
 0.27098468 0.1903272  0.135768  ]
[-0.60045663 -0.58711087 -0.57451055 -0.56089287 -0.54670889 -0.5327543
 -0.52091785 -0.51206265 -0.5045967  -0.49866349 -0.491622   -0.48400037
 -0.47147061 -0.4581822  -0.44004657 -0.42174343 -0.3951818  -0.36859468
 -0.33427121 -0.29994774 -0.25206812 -0.2041885  -0.12318155 -0.0421746
  0.0514483   0.14507121  0.19037151  0.2356718   0.27720649  0.31874117
  0.35463491  0.39052865  0.41509114  0.43965364  0.45118666  0.46271968
  0.46429482  0.46586996  0.46101521  0.45616045  0.4475746   0.43884145
  0.42879767  0.41783972  0.40688177  0.39553769  0.38351055  0.37148341
  0.35945626  0.34742912  0.33534688  0.32325281  0.31115873  0.29906465
  0.28697058  0.2748765   0.26402072  0.25375248  0.24348425  0.23321601
  0.22294777  0.21267954  0.2024113   0.19214307  0.18558077  0.17981465
  0.17404853  0.1682824   0.16251628  0.15675015]


In [24]:
# Slice before inter
print(di.u[0].sel(xu_ocean=190.,st_ocean=slice(10, 355))[:, 26].values)

[        nan -0.58711087 -0.57451055 -0.56089287 -0.54670889 -0.5327543
 -0.52091785 -0.51206265 -0.5045967  -0.49866349 -0.491622   -0.48400037
 -0.47147061 -0.4581822  -0.44004657 -0.42174343 -0.3951818  -0.36859468
 -0.33427121 -0.29994774 -0.25206812 -0.2041885  -0.12318155 -0.0421746
  0.0514483   0.14507121  0.19037151  0.2356718   0.27720649  0.31874117
  0.35463491  0.39052865  0.41509114  0.43965364  0.45118666  0.46271968
  0.46429482  0.46586996  0.46101521  0.45616045  0.4475746   0.43884145
  0.42879767  0.41783972  0.40688177  0.39553769  0.38351055  0.37148341
  0.35945626  0.34742912  0.33534688  0.32325281  0.31115873  0.29906465
  0.28697058  0.2748765   0.26402072  0.25375248  0.24348425  0.23321601
  0.22294777  0.21267954  0.2024113   0.19214307         nan         nan
         nan         nan         nan         nan]


# Calculating transport

In [11]:
# Area = metres in a degree of latitude x cell width x cell depth
area = LAT_DEG*0.1*5

# Multiply each grid cell by the constant.
dt = (di*area).sum(dim='yu_ocean').u.sel(st_ocean=slice(10, 355))
dt

In [19]:
T = 1
d3 = xr.open_dataset(dpath.joinpath('ofam_ocean_u_EUC_int_transport.nc'))

In [627]:
min_v = 0.1 # m/s
eps = np.round(min_v/2, 3)

for i, lon in enumerate(lons):
    du = [du_165, du_190, du_220][i]
    lon=lons[i]
    du = du.isel(time=slice(time_bnds_tao[i][0], time_bnds_tao[i][1])).sel(depth=slice(50, 355))
    umx, depths = lower_depths(du, i, min_v=min_v, eps=eps, index=True)
    tmp = np.ma.where(np.isnan(depths) == False)[0]
    xp = du.sel(depth=50).copy()*np.nan
    xp_165 = xp.copy()
    xp_190 = xp.copy()
    xp_220 = xp.copy()
    for t in tmp:
        r = du[t, 0:int(depths[t])+1]*5
        r = r.where(r > 0, 0)
        xp[t] = np.sum(r).item()
    if i == 0:
        xp_165 = xp
    elif i == 1:
        xp_190 = xp
    elif i == 2:
        xp_220 = xp

165: Min velocity=0.1, included=74, total=202, skipped=107, Empty=21, eps=0.05
190: Min velocity=0.1, included=167, total=296, skipped=88, Empty=41, eps=0.05
220: Min velocity=0.1, included=200, total=272, skipped=43, Empty=29, eps=0.05


In [619]:

t=15636501 
print(depths[t], du.depth[t].item(), du[t, int(depths[t])].item())
r = du[t, 0:int(depths[t])+1]*5
r = r.where(r > 0, 0)
np.sum(r).item()

41.0 125.0 0.08906452357769012


84.93547821044922

In [626]:
xp*230*1000/1e6

<xarray.DataArray 'u_1205' (time: 296)>
array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan, 24.04854  ,
       19.53516  , 22.334856 , 22.086304 ,        nan,        nan,
        5.3279314,  7.802441 , 10.47663  , 18.80938  , 23.878302 ,
       31.664022 , 29.151312 , 19.522327 , 15.455922 , 17.53414  ,
       18.130404 , 10.892059 ,  5.434436 , 10.715124 , 24.181532 ,
       24.1427   , 28.36451  , 19.023281 , 23.589392 , 17.431252 ,
       12.172214 ,  6.5705805,        nan, 10.84821  , 10.390587 ,
       10.881854 , 14.510607 , 16.015322 , 24.42754  , 22.541744 ,
       25.633165 , 21.182425 , 20.113651 , 15.146779 ,        nan,
       12.418961 ,  9.779358 ,        nan,        nan, 17.315077 ,
              nan, 21.203625 , 19.801292 , 20.027351 ,  9.334474 ,
       15.518783 , 12.924306 , 15.432197 , 11.831008 ,        nan,
              nan,    